In [69]:
import pandas as pd
import requests

In [70]:
url = "https://es.wikipedia.org/wiki/Poblaci%C3%B3n_mundial"

In [71]:
r = requests.get(url)

In [72]:
html_doc = r.text

In [73]:
html_doc

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="es" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Población mundial - Wikipedia, la enciclopedia libre</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"aa7e57e1-7119-4fd4-b931-2311d74fe3c3","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Población_mundial","wgTitle":"Población mundial","wgCurRevisionId":126462185,"wgRevisionId":126462185,"wgArticleId":373818,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Páginas con enlaces mágicos de ISBN","Wikipedia:Páginas con plantillas con argumentos duplicados","Wi

In [74]:
df_lista = pd.read_html(r.text) # Parsea todas las tablas de la url y las inserta en una lista, cada elemento es un df

In [75]:
df = df_lista[0]

In [76]:
df

,Continente,Densidad(hab./km²),Superficie(km²),Población(2019),País más poblado(2019),Ciudad más poblada(2019)
0,Asia,8688,43.810.000,4.677.291.000,China (1.392.240.000),Tokio (35.682.000)
1,África,327,30.370.000,1.110.020.000,Nigeria (191.205.000),El Cairo (16.794.000)
2,Europa,70,10.180.000,801.000.000,Rusia (112.000.000 Europa),Moscú (18.940.000)
3,América,235,42.330.000,1.094.215.000,Estados Unidos (330.028.000),Ciudad de México (22.577.000)[10]​
4,Oceanía,425,9.008.500,40.201.000,Australia (27.240.000),Sídney (6.550.000)
5,Antártida,"0,0003(varía)",13.720.000,"4.490(no permanente, varía)[11]​",N.D.[nota 1]​,N.D.


# Web Scraping con Beautiful Soup

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(html_doc)

In [ ]:
class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [ ]:
hp = HTMLTableParser()

In [ ]:
table = hp.parse_url(url)

In [ ]:
table[0][1].head()

# Ejercicio

In [77]:
url_ej="https://en.wikipedia.org/wiki/List_of_countries_by_average_wage"

In [78]:
r = requests.get(url_ej)

In [79]:
r.text

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of countries by average wage - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"55bec9df-423b-498b-b522-ec6d171495e0","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_countries_by_average_wage","wgTitle":"List of countries by average wage","wgCurRevisionId":959625455,"wgRevisionId":959625455,"wgArticleId":24486462,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Articles with tables with fixed row numbers","Lists by economic in

In [80]:
df_lista_ej = pd.read_html(r.text)

In [88]:
df1 = df_lista_ej[15]
df1

,Country,Averagemonthlywage
0,Switzerland,"$7,466"
1,Luxembourg,"$5,583"
2,Norway,"$5,418"
3,Denmark,"$5,310"
4,United States,"$4,893"
5,Australia,"$4,700"
6,Ireland,"$4,379"
7,Netherlands,"$4,289"
8,Canada,"$4,134"
9,Belgium,"$3,963"
